In [88]:
%%writefile kpiece.cpp



#include <chrono>

#include <iostream>
#include <vector>
#include <set>
#include <map>
#include <cmath>
#include <cstdlib>
#include <ctime>
#include <random>
#include <memory>
#include <algorithm>





// TreeNode Class
struct TreeNode : public std::enable_shared_from_this<TreeNode> {
    std::tuple<std::pair<int, int>, double, double> motion; // ((x, y), control, time)
    std::shared_ptr<TreeNode> parent;
    std::vector<std::shared_ptr<TreeNode>> children;

    TreeNode(std::tuple<std::pair<int, int>, double, double> motion, std::shared_ptr<TreeNode> parent = nullptr)
        : motion(motion), parent(parent) {}

    void addChild(std::tuple<std::pair<int, int>, double, double> childMotion) {
        auto child = std::make_shared<TreeNode>(childMotion, shared_from_this());
        children.push_back(child);
    }

    // Print the tree recursively
    void printTree(const std::string& prefix = "", bool isLast = true) const {
        auto [position, control, time] = motion;
        auto [x, y] = position;

        std::cout << prefix << (isLast ? "└── " : "├── ") << "(" << x << ", " << y << ") - t: " << time << ", u: " << control << "\n";
        for (size_t i = 0; i < children.size(); ++i) {
            children[i]->printTree(prefix + (isLast ? "    " : "│   "), i == children.size() - 1);
        }
    }

    // Get path to the goal
    std::vector<std::pair<int, int>> getPathToRoot() const {
        std::vector<std::pair<int, int>> path;
        auto current = shared_from_this();
        while (current) {
            path.push_back(std::get<0>(current->motion));
            current = current->parent;
        }
        std::reverse(path.begin(), path.end());
        return path;
    }
};

// Grid Class
class Grid {
private:
    int rows, cols;
    std::vector<std::vector<std::vector<std::tuple<std::pair<int, int>, double, double>>>> grid;
    std::set<std::pair<int, int>> obstacles;
    std::map<std::pair<int, int>, double> cellImportance;

public:
    Grid(int rows, int cols) : rows(rows), cols(cols) {
        grid.resize(rows, std::vector<std::vector<std::tuple<std::pair<int, int>, double, double>>>(cols));
    }

    void addObstacle(int x, int y) {
        if (x >= 0 && x < rows && y >= 0 && y < cols) {
            obstacles.insert({x, y});
        }
    }

    bool isObstacle(const std::pair<int, int>& position) const {
        return obstacles.find(position) != obstacles.end();
    }

    void addMotion(const std::tuple<std::pair<int, int>, double, double>& motion) {
        auto [position, _, __] = motion;
        auto [x, y] = position;
        grid[x][y].push_back(motion);
    }

    std::pair<int, int> selectCell(double bias) {
        std::vector<std::pair<int, int>> candidates;

        for (int x = 0; x < rows; ++x) {
            for (int y = 0; y < cols; ++y) {
                if (!grid[x][y].empty()) {
                    candidates.emplace_back(x, y);
                }
            }
        }

        if (candidates.empty()) return {-1, -1};

        if (static_cast<double>(rand()) / RAND_MAX < bias) {
            return *std::max_element(candidates.begin(), candidates.end(),
                                     [this](auto& a, auto& b) {
                                         return grid[a.first][a.second].size() < grid[b.first][b.second].size();
                                     });
        } else {
            return candidates[rand() % candidates.size()];
        }
    }

    std::tuple<std::pair<int, int>, double, double> selectMotion(const std::pair<int, int>& cell) {
        auto& motions = grid[cell.first][cell.second];
        if (motions.empty()) return {{-1, -1}, 0.0, 0.0};
        std::random_device rd;
        std::mt19937 gen(rd());
        std::normal_distribution<> d(0, motions.size() / 3.0);
        size_t idx = std::max<size_t>(0, std::min<size_t>(static_cast<size_t>(std::abs(d(gen))), motions.size() - 1));
        return motions[idx];
    }
};

// Helpers
bool isValidMotion(const std::pair<int, int>& position, const Grid& grid) {
    auto [x, y] = position;
    return x >= 0 && x < 60 && y >= 0 && y < 60 && !grid.isObstacle(position);
}

bool isGoalReached(const std::pair<int, int>& position) {
    return position == std::make_pair(50, 50);
}

std::tuple<std::pair<int, int>, double, double> expandMotion(const std::tuple<std::pair<int, int>, double, double>& motion,
                                                             Grid& grid, std::set<std::pair<int, int>>& visited) {
    std::vector<std::pair<int, int>> moves = {{1, 0}, {-1, 0}, {0, 1}, {0, -1},
                                              {1, 1}, {1, -1}, {-1, 1}, {-1, -1}};
    auto [position, _, time] = motion;
    auto [x, y] = position;

    for (const auto& [dx, dy] : moves) {
        auto newPosition = std::make_pair(x + dx, y + dy);
        if (isValidMotion(newPosition, grid) && visited.find(newPosition) == visited.end()) {
            visited.insert(newPosition);
            double control = static_cast<double>(rand()) / RAND_MAX;
            double newTime = time + static_cast<double>(rand()) / RAND_MAX;
            return {newPosition, control, newTime};
        }
    }
    return {{-1, -1}, 0.0, 0.0}; // Invalid motion
}

// KPIECE Implementation
std::shared_ptr<TreeNode> KPIECE(int iterations, const std::tuple<std::pair<int, int>, double, double>& initialMotion,
                                  Grid& grid, double bias = 0.75) {
    auto root = std::make_shared<TreeNode>(initialMotion);
    std::set<std::pair<int, int>> visited;
    visited.insert(std::get<0>(initialMotion));
    grid.addMotion(initialMotion);

    for (int i = 0; i < iterations; ++i) {
        auto cell = grid.selectCell(bias);
        if (cell == std::make_pair(-1, -1)) continue;

        auto motion = grid.selectMotion(cell);
        if (std::get<0>(motion) == std::make_pair(-1, -1)) continue;

        auto newMotion = expandMotion(motion, grid, visited);
        if (std::get<0>(newMotion) != std::make_pair(-1, -1)) {
            root->addChild(newMotion);
            grid.addMotion(newMotion);
            if (isGoalReached(std::get<0>(newMotion))) {
                std::cout << "Goal reached at: (" << std::get<0>(newMotion).first << ", " << std::get<0>(newMotion).second << ")\n";
                return root;
            }
        }
    }

    std::cout << "No solution found within the iteration limit.\n";
    return root;
}

int main() {
    srand(static_cast<unsigned>(time(0)));


    // Grid creation

    Grid grid(100, 100);
    grid.addObstacle(1, 2);
    grid.addObstacle(1, 0);
    grid.addObstacle(1, 2);

    auto initialMotion = std::make_tuple(std::make_pair(0, 0), 0.0, 0.0);

    // Define the goal position
    std::pair<int, int> goal = {50, 50};

    // Print Configuration Space
    std::cout << "Configuration Space (1 = obstacle, S = start, G = goal, 0 = free):\n";
    for (int i = 0; i < 100; ++i) {
        for (int j = 0; j < 100; ++j) {
            if (grid.isObstacle({i, j})) {
                std::cout << "1 ";
            } else if (std::make_pair(i, j) == std::make_pair(0, 0)) {
                std::cout << "S ";
            } else if (std::make_pair(i, j) == goal) {
                std::cout << "G ";
            } else {
                std::cout << "0 ";
            }
        }
        std::cout << "\n";
    }

    auto treeRoot = KPIECE(100000, initialMotion, grid);

    // Print the entire tree
    std::cout << "\nTree:\n";
    treeRoot->printTree();

    auto start_time = std::chrono::high_resolution_clock::now();
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end_time - start_time;
    std::cout << "KPIECE algorithm took " << elapsed.count() << " seconds.\n";




    return 0;
}



Overwriting kpiece.cpp


In [89]:
!g++ kpiece.cpp -o kpiece
!./kpiece

Configuration Space (1 = obstacle, S = start, G = goal, 0 = free):
S 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0